In [27]:
import pandas as pd
import numpy as np
import time
import keplergl as kgl
from dksr import extract_sample_network
from geojson import LineString as geoLS
from shapely.geometry import LineString as shapLS
from shapely.ops import unary_union
from shapely import MultiPoint

In [28]:
#north, east, south, west = 33.798, -84.378, 33.763, -84.422 

In [29]:
#berlin
north, east, south, west = 52.460298,13.328991,52.515803,13.455849

In [30]:
df = extract_sample_network(north, east, south, west, 50, seed=42)

In [31]:
df.head(10)#.style.background_gradient(cmap='Blues', subset='length_km')

,coordinates,length_km
0,"[[52.479385, 13.4398505], [52.4792742, 13.4392...",3.287428
1,"[[52.478999, 13.3623879], [52.4785928, 13.3633...",2.106747
2,"[[52.4859791, 13.3561118], [52.4868623, 13.357...",0.687891
3,"[[52.4844619, 13.3858234], [52.4843087, 13.384...",5.939523
4,"[[52.4837392, 13.4358193], [52.4829385, 13.437...",3.354199
5,"[[52.4989028, 13.3951472], [52.4989419, 13.395...",3.658486
6,"[[52.4858086, 13.3369571], [52.4859013, 13.335...",5.385234
7,"[[52.4681133, 13.4276733], [52.4674351, 13.427...",6.828486
8,"[[52.465742, 13.3899696], [52.4658311, 13.3893...",10.365038
9,"[[52.4947957, 13.4328748], [52.4938935, 13.428...",10.682533


In [32]:
geo_list = pd.DataFrame(np.zeros((len(df)),dtype=object),columns=['geo_json'])
                            
for i in range(0,len(df)):                       
    z_list = [0] * len(df['coordinates'][i])
    list0 = df['coordinates'][i]
    list1 = np.insert(list0,2,z_list,axis=1)
        
    geo_list.iloc[i] = [geoLS(list1.tolist())]

In [33]:
geo_list.iloc[0]

geo_json    {'type': 'LineString', 'coordinates': [[52.479...
Name: 0, dtype: object

In [34]:
#OSMnx returns values in lat, lng while KeplerGL expetcs them in lng,lat
def switch_lat_lon(coords):
    return [coords[1], coords[0], coords[2]]

In [35]:
# Apply the function to the 'coordinates' column using a lambda function
geo_list['geo_json'] = geo_list['geo_json'].apply(lambda x: {'type': x['type'], 'coordinates': [switch_lat_lon(coord) for coord in x['coordinates']]})

In [36]:
from IPython.display import display
with pd.option_context('display.max_colwidth', None):
    display(geo_list.iloc[1])

geo_json    {'type': 'LineString', 'coordinates': [[13.362388, 52.478999, 0.0], [13.363363, 52.478593, 0.0], [13.363443, 52.48138, 0.0], [13.36348, 52.483563, 0.0], [13.363639, 52.485183, 0.0], [13.363406, 52.485325, 0.0], [13.361487, 52.486046, 0.0], [13.360472, 52.486437, 0.0], [13.360283, 52.486328, 0.0], [13.358766, 52.485781, 0.0], [13.354191, 52.483484, 0.0], [13.352968, 52.484352, 0.0], [13.353241, 52.484494, 0.0], [13.353003, 52.484651, 0.0], [13.352731, 52.484518, 0.0]]}
Name: 1, dtype: object

In [37]:
#a first test plot
#map_0 = kgl.KeplerGl(height=800,data={'Scooters': geo_list})
#map_0

In [38]:
geo_list.iloc[0]

geo_json    {'type': 'LineString', 'coordinates': [[13.439...
Name: 0, dtype: object

In [39]:
# further transform routes data
# 1. interpolate the various LineStrings by shapely LineString interoplate function
# 2. generate timestamps for every interpolated point on a LineString

In [65]:
def trace_transform(df, speed, distance_delta):
    new_df = pd.DataFrame(columns=['coordinates', 'length_km', 'timestamps_list'])
    
    for index, row in df.iterrows():
        coordinates = row['coordinates']
        length_km = row['length_km']

        # create points by given distance_delta
        line = LineString(coordinates)
        distances = np.arange(0, line.length, distance_delta)
        points = MultiPoint([line.interpolate(distance) for distance in distances])
        lat_lon_values = [[p.x, p.y] for p in points.geoms]

        # partition_length in km
        total_points = len(lat_lon_values)
        partition_length = (length_km / total_points)

        # time distance in seconds
        time_delta = (partition_length / speed) * 3600

        row_timestamps = []

        for i in range(total_points):
            if i == 0:
                # set initial timestamp to current time
                row_timestamps.append(time.time())
            else:
                # calculate the time delta from the previous point
                time_delta = (partition_length / speed) * 3600
                # add the time delta to the previous timestamp
                prev_timestamp = row_timestamps[-1]
                next_timestamp = prev_timestamp + time_delta
                # append the new timestamp to the list
                row_timestamps.append(next_timestamp)

        new_df = new_df.append({'coordinates': lat_lon_values, 'length_km': length_km, 'timestamps_list': row_timestamps},
                               ignore_index=True)

    return new_df

In [66]:
df = trace_transform(df=df, speed=15, distance_delta=0.0001)

C:\Users\LukasKoch\AppData\Local\Temp\ipykernel_23092\3419988068.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'coordinates': lat_lon_values, 'length_km': length_km, 'timestamps_list': row_timestamps},
C:\Users\LukasKoch\AppData\Local\Temp\ipykernel_23092\3419988068.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'coordinates': lat_lon_values, 'length_km': length_km, 'timestamps_list': row_timestamps},
C:\Users\LukasKoch\AppData\Local\Temp\ipykernel_23092\3419988068.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'coordinates': lat_lon_values, 'length_km': length_km, 'timestamps_list': row_timestamps},
C:\Users\LukasKoch\AppData\Local\T

In [71]:
df.head(5)

,coordinates,length_km,timestamps_list
0,"[[52.479385, 13.4398505], [52.479366097543874,...",3.287428,"[1683119998.5352836, 1683120000.5792804, 16831..."
1,"[[52.478999, 13.3623879], [52.47896053913386, ...",2.106747,"[1683119998.5572348, 1683120001.0236704, 16831..."
2,"[[52.4859791, 13.3561118], [52.486044226491295...",0.687891,"[1683119998.5632474, 1683120000.5765877, 16831..."
3,"[[52.4844619, 13.3858234], [52.48444819873453,...",5.939523,"[1683119998.5924797, 1683120000.688782, 168312..."
4,"[[52.4837392, 13.4358193], [52.48369442944266,...",3.354199,"[1683119998.6011925, 1683120000.5937862, 16831..."


In [68]:
geo_list = pd.DataFrame(np.zeros((len(df)),dtype=object),columns=['geo_json'])
                            
for i in range(0,len(df)):                       
    z_list = [0] * len(df['coordinates'][i])
    list0 = df['coordinates'][i]
    list1 = np.insert(list0,2,z_list,axis=1)
    list2 = np.insert(list1,3,df['timestamps_list'][i],axis=1)
        
    geo_list.iloc[i] = [geoLS(list2.tolist())]

In [73]:
geo_list['geo_json'] = geo_list['geo_json'].apply(lambda x: {'type': x['type'], 'coordinates': [switch_lat_lon(coord) for coord in x['coordinates']]})

In [74]:
# a second plot
map_1 = kgl.KeplerGl(height=800,data={'Scooters': geo_list})
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Scooters':                                              geo_json
0   {'type': 'LineString', 'c…